In [1]:
import logging
import threading
import json
import itertools
#import modin.pandas as pd
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import ibm_db
import shap

from pandas_profiling import ProfileReport
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions.db import Database
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.enginelog import EngineLogging
from iotfunctions.dbtables import FileModelStore
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.dbtables import FileModelStore, DBModelStore
from mmfunctions.anomaly import (ARIMAForecaster, SpectralAnomalyScoreExt)
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
from sklearn import metrics
import scipy as sp
import scipy.fftpack
import skimage as ski  
from skimage import util as skiutil # for nifty windowing
import pyod as pyod
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

In [2]:
db_schema=None
class DatabaseDummy:
    tenant_id = '###_IBM_###'
    db_type = 'db2'
    model_store = FileModelStore('./')
    def _init(self):
        return

db = DatabaseDummy()
        
print (db.tenant_id)

2021-03-26T12:11:42.425 INFO iotfunctions.dbtables.__init__ Init FileModelStore with path: ./
###_IBM_###


In [3]:
# Get stuff in


df_input_raw = pd.read_csv('./BP_DATA/IoTData.csv', index_col=False, parse_dates=['DATETIME'], comment='#')

df_input_raw = df_input_raw.rename(columns={'DATETIME':'timestamp'}).\
                drop(columns=['UMIDADE_REL','KW','KW2'])
temp_air = sp.signal.resample(df_input_raw['TEMP_AIR'].astype(np.float64), 212840)

df_input_raw['entity'] = 'demand'
#df_input_raw['entity']=df_input_raw['DEVICE_ID']
#df_input_raw['timestamp']=df_input_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_input_raw = df_input_raw.sort_values(by='timestamp').dropna()
df_input_raw['timestamp'] = pd.to_datetime(df_input_raw['timestamp'])
df_input = df_input_raw.set_index(['timestamp']).resample("T", label="right").ffill()

#df_input

df_input


2021-03-26T12:11:42.574 INFO numexpr.utils._init_num_threads Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-03-26T12:11:42.576 INFO numexpr.utils._init_num_threads NumExpr defaulting to 8 threads.


,TEMP_AIR,entity
timestamp,,
2019-09-15 00:00:00,18.7,demand
2019-09-15 00:01:00,18.7,demand
2019-09-15 00:02:00,18.7,demand
2019-09-15 00:03:00,18.7,demand
2019-09-15 00:04:00,18.7,demand
...,...,...
2020-08-31 19:56:00,19.0,demand
2020-08-31 19:57:00,19.0,demand
2020-08-31 19:58:00,19.0,demand


In [4]:
df_input['TEMP_AIR'] = df_input['TEMP_AIR'].astype(np.float64)

In [5]:
df_input

,TEMP_AIR,entity
timestamp,,
2019-09-15 00:00:00,18.7,demand
2019-09-15 00:01:00,18.7,demand
2019-09-15 00:02:00,18.7,demand
2019-09-15 00:03:00,18.7,demand
2019-09-15 00:04:00,18.7,demand
...,...,...
2020-08-31 19:56:00,19.0,demand
2020-08-31 19:57:00,19.0,demand
2020-08-31 19:58:00,19.0,demand


In [6]:
ll = 50

df_large_list = []
#df_raw = df_input_raw.reset_index().set_index(['timestamp'])
for i in range(ll):
    df_new = df_input.copy()
    df_new['entity'] = 'entity' + str(i)
    df_large_list.append(df_new)

df_large = pd.concat(df_large_list)

In [7]:
df_large = df_large.reset_index().set_index(['entity','timestamp'])

In [8]:
df_large['TEMP_AIR'] = df_large['TEMP_AIR'].astype(np.float64)
df_large.index.levels[0].name

'entity'

In [9]:

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}


EngineLogging.configure_console_logging(logging.DEBUG)

spsi = SpectralAnomalyScoreExt('TEMP_AIR', 12, 'Spectral', 'SpectralInv')
et = spsi._build_entity_type(columns = [Column('TEMP_AIR',Float())], **jobsettings)
spsi._entity_type = et
df_large = spsi.execute(df = df_large)


2021-03-26T12:11:53.827 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2021-03-26T12:11:53.829 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-26T12:11:53.830 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-26T12:11:53.831 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 8.3.1
2021-03-26T12:11:53.833 DEBUG iotfunctions.util.__init__ Starting trace
2021-03-26T12:11:53.834 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_SpectralAnomalyScoreExt_20210326111153
2021-03-26T12:11:53.834 DEBUG iotfunctions.util.__init__ auto_save None
2021-03-26T12:11:53.835 DEBUG iotfunctions.util.categorize_args categorizing arguments
2021-03-26T12:11:53.837 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
2021-03-26T12:11:53.838 DEBUG iotfunctions.metadata.__init__ Initializ

2021-03-26T12:12:07.158 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:07.160 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:07.642 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:07.646 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:07.848 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09-15 00:08:00', '2019-09-15 00:09:00',
               ...
               '2020-08-31 19:51:00', '2020-08-31 19:52:00',
               '2020-08-31 19:53:00', '2020-08-31 19:54:00',
               '2020-08-31 19:55:00', '2020-08-31 19:56:00',
               '2020-08-31 19:57:00', '2020-08-31 19:58:00',
               '2020

2021-03-26T12:12:12.388 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:12.414 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:12.416 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:12.609 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:12.619 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:12.979 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:12.984 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:13.169 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

2021-03-26T12:12:17.468 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:17.496 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:17.497 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:17.747 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:17.754 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:18.177 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:18.182 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:18.471 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

2021-03-26T12:12:22.836 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:22.861 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:22.862 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:23.088 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:23.092 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:23.481 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:23.485 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:23.690 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

2021-03-26T12:12:28.259 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:28.282 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:28.283 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:28.499 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:28.501 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:28.891 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:28.897 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:29.091 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

2021-03-26T12:12:33.452 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:33.478 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:33.479 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:33.725 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:33.728 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:34.126 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:34.130 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:34.310 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

2021-03-26T12:12:38.765 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:38.791 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:38.792 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:39.002 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:39.003 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:39.396 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:39.400 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:39.594 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

2021-03-26T12:12:43.929 DEBUG mmfunctions.anomaly.prepare_data Spectral: prepare Data
2021-03-26T12:12:43.955 DEBUG mmfunctions.anomaly._calc Module Spectral, Entity: entity, Input: TEMP_AIR, Windowsize: 12, Output: Spectral, Overlap: 6, Inputsize: 506641
2021-03-26T12:12:43.956 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-26T12:12:44.190 DEBUG mmfunctions.anomaly._calc Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-26T12:12:44.192 DEBUG mmfunctions.anomaly._calc ->
2021-03-26T12:12:44.563 DEBUG mmfunctions.anomaly._calc -->
2021-03-26T12:12:44.568 DEBUG mmfunctions.anomaly._calc --->
2021-03-26T12:12:44.771 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09

In [10]:
df_large.describe()

,TEMP_AIR,Spectral
count,2.533205e+07,2.533205e+07
mean,2.041075e+01,5.779350e-01
std,4.353046e+00,6.544427e-01
min,8.300000e+00,3.182341e-01
25%,1.740000e+01,3.182341e-01
50%,2.010000e+01,3.182341e-01
75%,2.310000e+01,3.182341e-01
max,3.570000e+01,3.142341e+00
